# Devoir 2
rédigé par Corey Ducharme et Olivier Sirois

## Préamble

In [118]:
using JuMP
using Clp

Considérons le problème d'allocation de la demande stochastique.

On défini nos variables  

$$ X = (x_1, x_2, x_3, x_4)  \\
B = (10, 7, 16, 6) \\
Y = \begin{pmatrix}y_{11} & y_{12} & y_{13} \\ y_{21} & y_{22} & y_{23} \\ y_{31} & y_{32} & y_{33} \\ y_{41} & y_{42} & y_{43}\end{pmatrix} \\
A = \begin{pmatrix}40 & 24 & 4 \\ 45 & 27 & 4.5 \\ 32 & 19.2 & 3.2 \\ 55 & 33 & 5.5\end{pmatrix} $$

où $x_i$ représente la capacité de production de l'usine i,

B le vector des coûts de production,

$y_{ij}$ la production de glace j dans l'usine i,

$a_{ij}$ le cout de production pour la glace j dans l'usine i.

Nous posons maintenant le problème d'optimisation.

\begin{align*}
    \min_x\ & BX^{\intercal} + E[Q(x)]\\
    \mbox{s.t. } & X \geq \hat{0} \\
     & BX^{\intercal} \leq 120 \\
     & Xe \geq 12
\end{align*}


où le problème de 2e étape est 

\begin{align*}
Q(x) = \min_y\ & e^{\intercal}A \circ Y e \\
\mbox{s.t. } & Ye \leq X^{\intercal} \\
& Y^{\intercal}e \geq (\xi, 3, 2)^{\intercal}
\end{align*}

ici $\circ$ est le produit d'Hadamard et les opérations d'inégalités ce font éléments par éléments entre deux vecteurs. $e$ est un vecteur colonne unitaire associée à la matrice le précédent.

In [119]:
# Define the variables
scenarios = 3
p = [0.3, 0.4, 0.3]
ξ = [3, 5, 7]
B = [10 7 16 6]
A = [40 24 4; 45 27 4.5; 32 19.2 3.2; 55 33 5.5]
theta = -Inf

-Inf

In [120]:
function master(theta)
    master = Model(solver=ClpSolver())
    
    @variable(master, x[1:4] >= 0)
    
    @constraint(master, sum(x[i]*B[i] for i=1:4) <= 120)
    @constraint(master, sum(x[i] for i = 1:4) >= 12)
    
    @objective(master, Min, sum(x[i]*B[i] for i=1:4) + theta)
    
    status = solve(master)
    
    return master, status, getvalue(x)
end

master (generic function with 1 method)

In [121]:
master(-Inf)

(Minimization problem with:
 * 2 linear constraints
 * 4 variables
Solver is ClpMathProg, :Optimal, [0.0, 0.0, 0.0, 12.0])

In [122]:
function secondstage(x::Vector, ξ::Int)
    m = Model(solver=ClpSolver())    

    @variable(m, y[1:4,1:3] >= 0)
    
    @constraintref artCons[1:7]

    #NOTE : est-ce que la formule du dual va quand meme bien fonctionner si on met notre contrainte comme 
    #       etant un 'greater then' au lieu d'un 'lesser then' (>= vs. <=) dans le cas de nos contraintes
    #       1,2,3 ??
    artCons[1] = @constraint(m, sum(y[i,1] for i=1:4) >= ξ)
    artCons[2] = @constraint(m, sum(y[i,2] for i=1:4) >= 3)
    artCons[3] = @constraint(m, sum(y[i,3] for i=1:4) >= 2)
    artCons[4] = @constraint(m, sum(y[1,j] for j=1:3) <= x[1])
    artCons[5] = @constraint(m, sum(y[2,j] for j=1:3) <= x[2])
    artCons[6] = @constraint(m, sum(y[3,j] for j=1:3) <= x[3])
    artCons[7] = @constraint(m, sum(y[4,j] for j=1:3) <= x[4])
    
    
    #for i=1:4
    #    @constraint(m, sum(y[i,j] for j=1:3) <= x[i])
    #end
    
    @objective(m, Min, sum(A[i,j]*y[i,j] for i=1:4, j=1:3))
    
    status = solve(m)
    
    dual = getdual(artCons)
    
    
    return m, status, getvalue(y), dual
end

secondstage (generic function with 1 method)

In [123]:
martificial, status, y, dual = secondstage([0,0,0,12], 3)

(Minimization problem with:
 * 7 linear constraints
 * 12 variables
Solver is ClpMathProg, :Optimal, [0.0 0.0 0.0; 0.0 0.0 0.0; 0.0 0.0 0.0; 3.0 3.0 2.0], [55.0, 33.0, 5.5, -15.0, -10.0, -23.0, 0.0])

In [124]:
print(martificial)

Min 40 y[1,1] + 24 y[1,2] + 4 y[1,3] + 45 y[2,1] + 27 y[2,2] + 4.5 y[2,3] + 32 y[3,1] + 19.2 y[3,2] + 3.2 y[3,3] + 55 y[4,1] + 33 y[4,2] + 5.5 y[4,3]
Subject to
 y[1,1] + y[2,1] + y[3,1] + y[4,1] ≥ 3
 y[1,2] + y[2,2] + y[3,2] + y[4,2] ≥ 3
 y[1,3] + y[2,3] + y[3,3] + y[4,3] ≥ 2
 y[1,1] + y[1,2] + y[1,3] ≤ 0
 y[2,1] + y[2,2] + y[2,3] ≤ 0
 y[3,1] + y[3,2] + y[3,3] ≤ 0
 y[4,1] + y[4,2] + y[4,3] ≤ 12
 y[i,j] ≥ 0 ∀ i ∈ {1,2,3,4}, j ∈ {1,2,3}


In [125]:
y

4×3 Array{Float64,2}:
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 3.0  3.0  2.0

In [126]:
dual

7-element Array{Float64,1}:
  55.0
  33.0
   5.5
 -15.0
 -10.0
 -23.0
   0.0

In [152]:
function optimalcut(x::Vector, ξ::Int, p::Float64, dual::Vector)
    h =  [ξ; 3; 2; 0; 0; 0; 0];
    T =  [0 0 0 0; 0 0 0 0; 0 0 0 0; -x[1] 0 0 0; 0 -x[2] 0 0; 0 0 -x[3] 0; 0 0 0 -x[4]]   
    small_e = p * dot(vec(dual), h)
    E = zeros(4)
    for i = 1:4
        E[i] = p* dot(vec(dual),T[:,i])
    end
    return small_e, E
end

#function feascut(x::Vector, ξ::Int)
    
#end

optimalcut (generic function with 4 methods)

In [155]:
e, E = optimalcut([0,0,0,12], 3, 0.3, dual)
@printf("petit e = %f;\n",a)
E

petit e = 82.500000;


4-element Array{Float64,1}:
 0.0
 0.0
 0.0
 0.0